In [ ]:
import sys
sys.path.append(r'C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools')

In [1]:
#%load load.py

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from pyexcel_ods import get_data as OpenOffice

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## In working with the ICD10 and ICD9 Lookup tables, it's best to have them as pickles

In [ ]:
## Read in all the data from .ods file
ooDoc = OpenOffice(r"C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools\src\ICD10_BETAX.ods")
print(ooDoc.keys())

In [ ]:
# ICD 10s
dfICD10s = pd.DataFrame(np.array(ooDoc['ICD10_BETAX'])[1:], columns=np.array(ooDoc['ICD10_BETAX'])[0] )

# ICD 9s
cols = np.array(ooDoc['ICD9_BetaX'])[0][:2]
dfICD9s = pd.Series(np.array(ooDoc['ICD9_BetaX'])[1:]).apply(lambda c: 'MARKREGINE'.join(c[:2])).str.split('MARKREGINE', expand=True).rename(columns={0:'ICD9', 1:'Description'})

del cols

In [ ]:
# ICD10_to_ICD9
dfICD10_to_ICD9 = pd.DataFrame(np.array(ooDoc['ICD10_to_ICD9'])[1:], columns=np.array(ooDoc['ICD10_to_ICD9'])[0] )

# ICD9_to_ICD10
dfICD9_to_ICD10 = pd.DataFrame(np.array(ooDoc['ICD9_to_ICD10'])[1:], columns=np.array(ooDoc['ICD9_to_ICD10'])[0] )

In [ ]:
# ICD9_to_ICD10
dfICD10_levels_123 = pd.DataFrame(np.array(ooDoc['ICD10_levels_123'])[1:], columns=np.array(ooDoc['ICD10_levels_123'])[0] )

# ICD10_levels_12345
dfICD10_levels_12345 = pd.DataFrame(np.array(ooDoc['ICD10_levels_12345'])[1:], columns=np.array(ooDoc['ICD10_levels_12345'])[0] )

In [ ]:
del ooDoc

In [ ]:
dfICD10s.to_pickle(r"C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools\src\dfICD10LookUp.pkl")
dfICD9s.to_pickle(r"C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools\src\dfICD9LookUp.pkl")
dfICD10_to_ICD9.to_pickle(r"C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools\src\dfICD10_to_ICD9.pkl")
dfICD9_to_ICD10.to_pickle(r"C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools\src\dfICD9_to_ICD10.pkl")

dfICD10_levels_123.to_pickle(r"C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools\src\dfICD10_levels_123.pkl")
dfICD10_levels_12345.to_pickle(r"C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools\src\dfICD10_levels_12345.pkl")

## --------------------------------------------------------------------------

## Build ICD Helper tables in HAT

## --- Prep ICD 9 Lookup ---

In [ ]:
dfICD9LookUp = pd.read_pickle(r"C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools\src\dfICD9LookUp.pkl")

In [ ]:
dfICD9Lookup = dfICD9Lookup.apply(lambda c: c.str.strip()).copy()

In [ ]:
print("Is the ICD 9 Code in dfICD9Lookup unique:\n", dfICD9Lookup.set_index('ICD9').index.is_unique )

## --- Prep ICD 10 Lookup ---

In [ ]:
dfICD10LookUp = pd.read_pickle(r"C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools\src\dfICD10LookUp.pkl")

In [ ]:
dfICD10Lookup = dfICD10Lookup.apply(lambda c: c.str.strip()).copy()

In [ ]:
print("Is the ICD 10 Code in dfICD10Lookup unique:\n", dfICD10Lookup.set_index('ICD10').index.is_unique )

## --- Prep ICD9 to ICD10 ---

In [ ]:
dfICD9_to_ICD10 = pd.read_pickle(r"C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools\src\dfICD9_to_ICD10.pkl")

* #### Dtype and strip

In [3]:
dfICD9_to_ICD10 = dfICD9_to_ICD10.apply(lambda c: c.str.strip()).copy()

* #### Add Extra zeros to ICD 9 codes

In [4]:
print("Make sure all ICD9s have a length of 5:\n", dfICD9_to_ICD10['ICD9'].str.len().value_counts())

dfICD9_to_ICD10['ICD9'] = \
    np.where(dfICD9_to_ICD10['ICD9'].str.len() == 5, dfICD9_to_ICD10['ICD9'], 
        np.where(dfICD9_to_ICD10['ICD9'].str.len() == 4, dfICD9_to_ICD10['ICD9'] + '0', 
            np.where(dfICD9_to_ICD10['ICD9'].str.len() == 3, dfICD9_to_ICD10['ICD9'] + '00', 'MarkRegine')))

print("Make sure all ICD9s have a length of 5:\n", dfICD9_to_ICD10['ICD9'].str.len().value_counts())

Make sure all ICD9s have a length of 5:
 5    16691
4     7111
3      145
Name: ICD9, dtype: int64
Make sure all ICD9s have a length of 5:
 5    23947
Name: ICD9, dtype: int64


* #### One ICD9 Code Linkes to More than one ICD10

In [5]:
t = dfICD9_to_ICD10[dfICD9_to_ICD10.duplicated('ICD9', keep=False)]
print(t.head())
print(t['ICD9'].nunique())

      ICD9  ICD10   FLAG
30   00589   A054  10000
31   00589   A058  10000
284  01480  A1832  10000
285  01480  A1839  10000
286  01481  A1832  10000
3188


In [6]:
counts = t.groupby('ICD9').apply(lambda g: g.reset_index())
counts.index = counts.index.get_level_values(level=1)
counts.index.name = 'ICD10_Addition'
dfICD9_to_10_OneRecPer9 = counts.reset_index().pivot(index='ICD9', columns='ICD10_Addition', values='ICD10')

* #### Result is that one ICD9 can have up-to 533 ICD10s

In [7]:
print(dfICD9_to_10_OneRecPer9.index.is_unique)
dfICD9_to_10_OneRecPer9.head()

True


ICD10_Addition,0,1,2,3,4,5,6,7,8,9,...,523,524,525,526,527,528,529,530,531,532
ICD9,,,,,,,,,,,,,,,,,,,,,
00589,A054,A058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01480,A1832,A1839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01481,A1832,A1839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01482,A1832,A1839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01483,A1832,A1839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## ---- Prep ICD10 to ICD9 ----

In [ ]:
dfICD10_to_ICD9 = pd.read_pickle(r"C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools\src\dfICD10_to_ICD9.pkl")

* #### Dtype and strip

In [10]:
dfICD10_to_ICD9 = dfICD10_to_ICD9.apply(lambda c: c.str.strip()).copy()

* #### Add Extra zeros to ICD9 codes

In [11]:
print("Make sure all ICD9s have a length of 5:\n", dfICD10_to_ICD9['ICD9'].str.len().value_counts())

dfICD10_to_ICD9['ICD9'] = \
    np.where(dfICD10_to_ICD9['ICD9'].str.len() == 5, dfICD10_to_ICD9['ICD9'], 
        np.where(dfICD10_to_ICD9['ICD9'].str.len() == 4, dfICD10_to_ICD9['ICD9'] + '0', 
            np.where(dfICD10_to_ICD9['ICD9'].str.len() == 3, dfICD10_to_ICD9['ICD9'] + '00', 'MarkRegine')))

print("Make sure all ICD9s have a length of 5:\n", dfICD10_to_ICD9['ICD9'].str.len().value_counts())

Make sure all ICD9s have a length of 5:
 5    53869
4    25067
3      256
Name: ICD9, dtype: int64
Make sure all ICD9s have a length of 5:
 5    79192
Name: ICD9, dtype: int64


* #### Yes, this table too has ICD10s linking to more then one ICD9

In [12]:
print(dfICD10_to_ICD9['ICD10'].is_unique)
dfICD10_to_ICD9[dfICD10_to_ICD9.duplicated('ICD10', keep=False)].head()

False


,ICD10,ICD9,FLAG
15,A021,00310,10111
16,A021,99591,10112
36,A044,00800,10000
37,A044,00809,10000
41,A048,00820,10000


## ------------------------------------------------------------------------------------------------------------------

## --- Extract_Dx_Regardless_of_ICD_Version ---

In [ ]:
def Extract_Dx_Regardless_of_ICD_Version(df, list_of_icds_to_search_for, Version=9, DxNumber='All'):
    """ ---------------------------------  
    This will search the medical df for ICDs of interest regardless of ICD version used.  
    Required that the dfMed table has a column (col name 'ICDversion' of type str) indicating the ICDversion == '9'|'0'  
    Currently this only uses the fields Dx1, Dx2, Dx3 (see function code). Ammend function if needed.  
    Dependancies: dfICD9_to_ICD10 & dfICD10_to_ICD9
    Returns a DataFrame. 
    ----  
    df: dfMed.  
    list_of_icds_to_search_for: ex. ['25000', '49300'].  
    Version: The version of ICD codes listed above.  
    DxNumber: primary('1'), secondary('2'), tertiary('3'), and '12', ...  
    -------------------------------------  
    """
    ## Takes the medical and melts it
    t = df[['ICDversion', 'Dx1', 'Dx2', 'Dx3']].reset_index(drop=False).copy()  ## IF you change to include all pd.Series(dfMed.columns).str.startswith Dx[0-9] then you have to change below too!
    t.rename(columns={'index':'index_orig'}, inplace=True)
    tt = pd.melt(t, id_vars=['index_orig', 'ICDversion'], value_vars=['Dx1', 'Dx2', 'Dx3']).copy()
    print("this should be == to the number of dx columns and then the number of records times the number of dx cols:\n",  tt['index_orig'].value_counts().sort_values().value_counts())
    print("vc of IDCversions in the data set:\n", tt['ICDversion'].value_counts())

    if Version == 9:
        ## if searching for an ICD9, look in ICD9s 
        df9s = tt.query("ICDversion == '9'").copy()
        if DxNumber == 'All':
            index_values = df9s.loc[df9s.value.isin(list_of_icds_to_search_for), 'index_orig']  
        elif DxNumber == '1':
            index_values = df9s.loc[(df9s.value.isin(list_of_icds_to_search_for)) & (df9s['variable'] == 'Dx1'), 'index_orig'] 
        elif DxNumber == '12':
            index_values = df9s.loc[(df9s.value.isin(list_of_icds_to_search_for)) & ((df9s['variable'] == 'Dx1') | (df9s['variable'] == 'Dx2')), 'index_orig']            
        r1 = df.loc[index_values, :]

        ## if searching for an ICD9, look in ICD10s 
        # return a list of ICD10s that link to the ICD9 searching for
        ICD10s_to_find_in_medical = dfICD9_to_ICD10.loc[dfICD9_to_ICD10.ICD9.str.strip().str.contains('|'.join(list_of_icds_to_search_for)), 'ICD10']
        print(ICD10s_to_find_in_medical)
        # select recs from med where verison is 10
        df10s = tt.query("ICDversion == '0'").copy()
        if DxNumber == 'All':        
            index_values = df10s.loc[df10s.value.isin(ICD10s_to_find_in_medical), 'index_orig']
        elif DxNumber == '1':
            index_values = df10s.loc[(df10s.value.isin(ICD10s_to_find_in_medical)) & (df10s['variable'] == 'Dx1'), 'index_orig']
        elif DxNumber == '12':
            index_values = df10s.loc[(df10s.value.isin(ICD10s_to_find_in_medical)) & ((df10s['variable'] == 'Dx1') | (df10s['variable'] == 'Dx2')), 'index_orig'] 
        r2 = df.loc[index_values, :]
        r = pd.concat([r1, r2])
        return r
    
    if Version == 10:
        ## if searching for an ICD10, look in ICD10s 
        df10s = tt.query("ICDversion == '0'").copy()
        if DxNumber == 'All':
            index_values = df10s.loc[df10s.value.isin(list_of_icds_to_search_for), 'index_orig']  
        elif DxNumber == '1':
            index_values = df10s.loc[(df10s.value.isin(list_of_icds_to_search_for)) & (df10s['variable'] == 'Dx1'), 'index_orig'] 
        elif DxNumber == '12':
            index_values = df10s.loc[(df10s.value.isin(list_of_icds_to_search_for)) & ((df10s['variable'] == 'Dx1') | (df10s['variable'] == 'Dx2')), 'index_orig']            
        r1 = df.loc[index_values, :]
        
        ## if searching for an ICD10, look in ICD9s 
        # return a list of ICD9s that link to the ICD10 searching for
        ICD9s_to_find_in_medical = dfICD10_to_ICD9.loc[dfICD10_to_ICD9['ICD10'].str.strip().isin(list_of_icds_to_search_for), 'ICD9']
        print(ICD9s_to_find_in_medical)
        # select recs from med where verison is 9
        df9s = tt.query("ICDversion == '9'").copy()
        if DxNumber == 'All':        
            index_values = df9s.loc[df9s.value.isin(ICD9s_to_find_in_medical), 'index_orig']
        elif DxNumber == '1':
            index_values = df9s.loc[(df9s.value.isin(ICD9s_to_find_in_medical)) & (df9s['variable'] == 'Dx1'), 'index_orig']
        elif DxNumber == '12':
            index_values = df9s.loc[(df9s.value.isin(ICD9s_to_find_in_medical)) & ((df9s['variable'] == 'Dx1') | (df9s['variable'] == 'Dx2')), 'index_orig'] 
        r2 = df.loc[index_values, :]
        r = pd.concat([r1, r2])
        return r 